In [ ]:
import cv2
import numpy as np

import os
from os import listdir
from os.path import isfile, join
import yaml


model_folder = "../Models/beachbot_yolov5s_trash_detection__minimal_example__yolov5pytorch_160/"
dataset_location = "../Datasets/trash_detection.minimal_example.yolov5pytorch/"



## Read image size required by exported model:
* must be multiple of 32

In [ ]:

with open(model_folder + "/export_info.yaml", 'r') as stream:
    export_info = yaml.safe_load(stream)
    img_height = export_info['img_heigt_export']
    img_width = export_info['img_width_export']

print("Exported ONNX model operates in omages of size ", img_width, "x", img_height, "[wxh] pixels")



## Read dataset class names and define helper functions:

In [ ]:

train_folder=dataset_location + "/train"
test_folder=dataset_location + "/valid"

# read number of classes based on YAML config file:

with open(dataset_location + "/data.yaml", 'r') as stream:
    data_cfg = yaml.safe_load(stream)
    num_classes = str(data_cfg['nc'])
    list_classes = data_cfg['names']
    print("Dataset defines", num_classes, "classes ->\n", list_classes)
    
train_imgs = [f for f in listdir(train_folder+"/images") if (isfile(join(train_folder+"/images", f)) and f.endswith(".jpg"))]
test_imgs = [f for f in listdir(test_folder+"/images") if (isfile(join(test_folder+"/images", f)) and f.endswith(".jpg"))]


# function to get the output layer names 
# in the architecture
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

## Read iamge, rescale and format to fit imported ONNX model:

In [ ]:


# read input image
image = join(train_folder+"/images", train_imgs[0])
image = cv2.imread(image)


# Width = image.shape[1]
# Height = image.shape[0]
Width = img_width
Height =  img_height



# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(list_classes), 3))

# read pre-trained model and config file
model_cfg_file="?"
for file in os.listdir(model_folder):
    if file.endswith(".yaml"):
        model_cfg_file = model_folder + str(file)
        break
print(model_cfg_file)
net = cv2.dnn.readNet(model_folder + "best.onnx", model_cfg_file)

is_cuda = False
if is_cuda:
    print("Attempty to use CUDA")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
else:
    print("Running on CPU")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)


## Apply model and visualize results (draw boxes etc):

In [ ]:



# Reformat image format to fit for yolo model
def format_yolov5(frame):
    row, col, _ = frame.shape
    _max = max(col, row)
    result = np.zeros((_max, _max, 3), np.uint8)
    result[0:row, 0:col] = frame
    return result

# Run inference
def detect(image, net):
    # create input blob, convert color and value range of input image to fit to yolo model:
    scale = 1.0/255.0 # convert byte color 0-255 to float value range 0-1
    blob = cv2.dnn.blobFromImage(image, scale, (Width,Height), (0,0,0), True, crop=False)
    net.setInput(blob)
    
    # apply model:
    preds = net.forward()
    return preds


# Transform output into image coordinates (bos positions)
# Each proposed box has a confidence score, only process boxes that exceed a certain threshold
# (the model is shure the box and label are correct)
# There are actually confidence and score thresholds -> TODO understand what they mean :)
#
def wrap_detection(input_image, output_data):
    class_ids = []
    confidences = []
    boxes = []

    rows = output_data.shape[0]
    image_width, image_height, _ = input_image.shape

    x_factor = image_width / Width
    y_factor =  image_height / Height

    for r in range(rows):
        row = output_data[r]
        confidence = row[4]
        if confidence >= 0.2:

            classes_scores = row[5:]
            _, _, _, max_indx = cv2.minMaxLoc(classes_scores)
            class_id = max_indx[1]
            if (classes_scores[class_id] > .25):

                confidences.append(confidence)

                class_ids.append(class_id)

                x, y, w, h = row[0].item(), row[1].item(), row[2].item(), row[3].item() 
                left = int((x - 0.5 * w) * x_factor)
                top = int((y - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])
                boxes.append(box)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.45) 

    result_class_ids = []
    result_confidences = []
    result_boxes = []

    for i in indexes:
        result_confidences.append(confidences[i])
        result_class_ids.append(class_ids[i])
        result_boxes.append(boxes[i])

    return result_class_ids, result_confidences, result_boxes








# Apply model to image
inputImage = format_yolov5(image)
outs = detect(inputImage, net)

# Transform output into image coordinates (bos positions)
# Each proposed box has a confidence score, only process boxes that exceed a certain threshold
# (the model is shure the box and label are correct)
class_ids, confidences, boxes = wrap_detection(inputImage, outs[0])


# Draw boxes, visualize results
colors = [(255, 255, 0), (0, 255, 0), (0, 255, 255), (255, 0, 0), (255, 0, 0), (255, 0, 0), (255, 0, 0), (255, 0, 0), (255, 0, 0), (255, 0, 0)]
for (classid, confidence, box) in zip(class_ids, confidences, boxes):
    if confidence >= 0.2:
        color = colors[int(classid) % len(colors)]
        cv2.rectangle(image, box, color, 2)
        cv2.rectangle(image, (box[0], box[1] - 20), (box[0] + box[2], box[1]), color, -1)
        cv2.putText(image, list_classes[classid], (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,0))

        
from matplotlib import pyplot as plt
import numpy as np
import cv2
img2 = image[:,:,::-1]
plt.imshow(img2)

